In [24]:
# BeautifulSoup, Pandas, and Requests/Splinter
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
import requests

In [25]:
# Check for chrome web driver path
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
# Create Browser instance and pass 'chrome' string
def browser():
    executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
    return Browser('chrome', **executable_path, headless=False)
browser = browser()

In [4]:
# Assign news url to variable
news_url= "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(news_url)
news_html = browser.html
news_soup = BeautifulSoup(news_html, 'html.parser')

In [5]:
# Pretty print 
# print(news_soup.prettify())

In [6]:
# NASA Mars News Scraping

In [7]:
news_title = news_soup.body.find('div', class_='content_title').get_text()
print(news_title)

NASA's Mars 2020 Rover Does Biceps Curls 


In [8]:
news_p = news_soup.body.find('div', class_='article_teaser_body').get_text()
print(news_p)


In this time-lapse video, the robotic arm on NASA's Mars 2020 rover maneuvers its 88-pound (40-kilogram) sensor-laden turret as it moves from a deployed to stowed configuration.


In [9]:
# Set image browser
image_url= "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url)
image_html= browser.html
image_soup = BeautifulSoup(image_html, 'lxml')
# print(image_soup.prettify())


In [10]:
# Set base url 
base_url = 'https://www.jpl.nasa.gov'

# Get featured image partial url
image_url = image_soup.find("a", {"class":"button fancybox"})['data-fancybox-href']

featured_image_url = base_url + image_url
print(featured_image_url)


https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA08097_ip.jpg


In [11]:
# print(base_url)

In [12]:
# Mars Weather

In [13]:
# Set weather browser 
twitter_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(twitter_url)
weather_html = browser.html
weather_soup = BeautifulSoup(weather_html, 'lxml')
# print(weather_soup.prettify())                         

In [14]:
# Get latest tweet about weather from twitter link 
mars_weather = weather_soup.find('p', class_= "TweetTextSize").text.split(".")[0]
print(mars_weather)

InSight sol 233 (2019-07-23) low -98


In [15]:
# Mars Facts

In [16]:
# Scrape second HTML table with pandas
mars_facts_url = 'https://space-facts.com/mars/'
mars_facts_table = pd.read_html(mars_facts_url)
mars_facts_table = mars_facts_table[1]

In [17]:
# print(mars_facts_table)

In [18]:
# Read table to pandas DataFrame
mars_facts_df = mars_facts_table
mars_facts_df.columns = ['Description', 'Value']
mars_facts_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [19]:
# Convert to HTML table string
mars_facts_html = mars_facts_df.to_html()
print(mars_facts_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Description</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
    

In [20]:
# Mars Hemisphere Browser
hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars' 
browser.visit(hemisphere_url)
hemisphere_html = browser.html
hemisphere_soup = BeautifulSoup(hemisphere_html, 'lxml')

In [21]:
# Create list to hold list of links      
links = []  

# Create list to hold img urls
hemisphere_img_urls = []    

# Store base url in a variable for concatenation
base_url = "https://astrogeology.usgs.gov" 

result = hemisphere_soup.find('div', class_='result-list')
#print(result)

items = result.findAll('div', class_ = 'item')
#print(items)

#loop through divs
for item in items:
    # Create dictionary to append list of images
    hemisphere = {} 
    
    content = item.find('div', class_='description')
    
    title = content.find('h3').text
    hemisphere['title'] = title
    
    href = item.find('a', {"class":"itemLink product-item"})['href']
    links.append(base_url + href)
    
#loop through links
    for link in links:
        response = requests.get(link)
        hemisphere_soup = BeautifulSoup(response.text, 'lxml')

        img_src = hemisphere_soup.find("img", {"class":"wide-image"})['src']
        img_url = base_url + img_src
        hemisphere['img_url'] = img_url
        
    hemisphere_img_urls.append(hemisphere)
    
hemisphere_img_urls



[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]